In [310]:
# Import multiple librarys for Data request and cleanup

import requests # this library handles http requests... for eg API GET requests.
import os # enables me to retrieve api key from another file
from dotenv import load_dotenv # allows me to access env file with api key
import pandas as pd # for all things pandas
import pgeocode
 # no longer required from geopy.geocoders import Nominatim # to clean up location data
import json # to parse json
import numpy as np
from datetime import datetime

load_dotenv("store.env") # loads data from env file

True

In [295]:
versionnumber = "1.0"
keywords = "data"
locationName = "birmingham" # location search criterea
employerId = ""
distanceinmiles = "30"
resultsToTake = 100
resultsToSkip = 0
reed_api_key = os.getenv("reed_api_key") #retrieves api_key from .env file. This means i don't need to hard code my API key into script for improved security.

url = f"https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&locationName={locationName}&employerId={employerId}&distanceFromLocation={distanceinmiles}&resultsToTake={resultsToTake}&resultsToSkip={resultsToSkip}"
url

'https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=0'

In [296]:
response = requests.get(url, auth=(reed_api_key, ''))
json_for_df = [] # initialise list to extend reed_data to.
if response.status_code == 200:
    print(f"Response Success {response.status_code}") 
    reed_data = response.json() # get json and set it to reed data. Add reed data to json_for_df below
    json_for_df.extend(reed_data["results"])
    resultsToSkip = resultsToSkip + resultsToTake # increment results to skip so i don't append the same data in the loop below
    total_results = (reed_data["totalResults"])
else:
    raise Exception (f"Unsuccesfull API request: {response.status_code}") # stops execution of code and exits with error message.


Response Success 200


In [297]:
total_pages_minus_1 = ((total_results + resultsToTake - 1) // resultsToTake) - 1  # this gives the total number of pages minues 1 (because i already have the first API page) to retrieve data from.

for num_page in range(total_pages_minus_1):  # ie from 0 to the total number of pages
    print("-----")
    url = f"https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&locationName={locationName}&employerId={employerId}&distanceFromLocation={distanceinmiles}&resultsToTake={resultsToTake}&resultsToSkip={resultsToSkip}"
    response = requests.get(url, auth=(reed_api_key, ''))
    reed_data = response.json()
    json_for_df.extend(reed_data["results"])
    print("requesting:", url)
    resultsToSkip = resultsToSkip + resultsToTake # increment results to skip by 100 for every page

-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=100
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=200
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=300
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=400
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=500
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=data&locationName=birmingham&employerId=&distanceFromLocation=30&resultsToTake=100&resultsToSkip=600
----

In [298]:
df = pd.DataFrame(json_for_df)
# Data clean-up:
df.fillna("NaN", inplace=True) # this fills empty values with NaN
df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y") # turn the date into an actual date-time date

df = df.drop_duplicates(subset='jobId')

df



C:\Users\Henry\AppData\Local\Temp\ipykernel_2932\2639542874.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NaN", inplace=True) # this fills empty values with NaN


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl
0,54419415,500373,QA,NaN,NaN,Data Administrator Apprentice,CV12AQ,14000.0,16000.0,GBP,18/03/2025,2025-02-04,About Premier Solutions Ltd: Premier Solutions...,57,https://www.reed.co.uk/jobs/data-administrator...
1,54494579,666205,Stark Building Materials UK,NaN,NaN,HR Data & Process Support Specialist,Coventry,NaN,NaN,NaN,17/03/2025,2025-02-17,We are seeking a motivated and detail-oriented...,35,https://www.reed.co.uk/jobs/hr-data-process-su...
2,54599030,429392,Searchability,NaN,NaN,Data Scientist,Birmingham,45000.0,70000.0,GBP,17/04/2025,2025-03-06,"Data Scientist Key Highlights: Salary: 45,000 ...",29,https://www.reed.co.uk/jobs/data-scientist/545...
3,54405886,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham | Hybrid - 3 Days On...,69,https://www.reed.co.uk/jobs/data-scientist/544...
4,54404538,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham 3 days a week We are...,78,https://www.reed.co.uk/jobs/data-scientist/544...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414,54563954,674264,Appcast Enterprise,NaN,NaN,Litigation Handler/Solicitor,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/litigation-handler...
2415,54563961,674264,Appcast Enterprise,NaN,NaN,Senior Underwriter,Birmingham,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Gallagher isn't just a company; i...,0,https://www.reed.co.uk/jobs/senior-underwriter...
2416,54563922,674264,Appcast Enterprise,NaN,NaN,Litigation Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,1,https://www.reed.co.uk/jobs/litigation-handler...
2417,54563259,674264,Appcast Enterprise,NaN,NaN,Motor Claims Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/motor-claims-handl...


In [299]:
'''
geolocator = Nominatim(user_agent='postcode_checker')

# this function checks whether the postcode is a string & if it contains a number. If it contains a number it's likely a postcode so pass the str into the next function
def is_postcode(value):
    return isinstance(value, str) and any(character.isdigit() for character in value)


# The following function uses the postcode past into get_town & returns the 3rd from last value in the returned list. Which is often the town
def get_display_name(value):
    if is_postcode(value):
        location = geolocator.geocode(value, exactly_one=True)
        if location:
            display_name = location.raw.get("display_name", "")
            return display_name
        else:
            return value
    return value

# The following function parses the display_name from the comma and returns the -3 item in the list, if it is longer than 3 items long, as this is usually the town. If not then just returns the original value.
def get_town(value):
    value = value.split(", ")
    if len(value) >= 3:
        return value[-3]  
    else:
        return value[0]


# The following function also parses the display_name from the comma and returns the -1 item which is usually the country. If not then returns NaN
def get_country(value):
    value = value.split(", ")
    if len(value) >= 3:
        return value[-1]  
    else:
        return "NaN"
'''

'\ngeolocator = Nominatim(user_agent=\'postcode_checker\')\n\n# this function checks whether the postcode is a string & if it contains a number. If it contains a number it\'s likely a postcode so pass the str into the next function\ndef is_postcode(value):\n    return isinstance(value, str) and any(character.isdigit() for character in value)\n\n\n# The following function uses the postcode past into get_town & returns the 3rd from last value in the returned list. Which is often the town\ndef get_display_name(value):\n    if is_postcode(value):\n        location = geolocator.geocode(value, exactly_one=True)\n        if location:\n            display_name = location.raw.get("display_name", "")\n            return display_name\n        else:\n            return value\n    return value\n\n# The following function parses the display_name from the comma and returns the -3 item in the list, if it is longer than 3 items long, as this is usually the town. If not then just returns the original va

In [300]:
# retrieve UK postcode data from pgeocode API and save in a df. ill do a merge between main df & geo_df to get location data
country = "GB"

download_url = f"https://symerio.github.io/postal-codes-data/data/geonames/{country}.txt"

column_names = [
    "countryCode",
    "outwardCode", 
    "nonPostcodeValues", 
    "countryName", 
    "countryAbv", 
    "countyName", 
    "admin_1", 
    "districtName", 
    "onsCode", 
    "latitude", 
    "longitude", 
    "accuracy"
]

response = requests.get(download_url)

if response.status_code == 200:
    print(f'Response sucess {response.status_code}')
    df_geo = pd.read_csv(download_url, sep="\t", header=None, names=column_names, dtype=str)
    
else:
    print(f'Unsuccesful {response.status_code}')

# drop duplicate rows for outward_code to avoid multiple matches at the join stage:
df_geo_unique = df_geo.drop_duplicates(subset='outwardCode')
# drop unwanted columns from df_geo_unique:
df_geo_unique = df_geo_unique.drop(columns=['countryAbv', 'admin_1', 'districtName', 'onsCode', 'latitude', 'longitude', 'accuracy'])

Response sucess 200


In [301]:
def postcode_breakdown (value): # this function finds postcodes (if contains digit assume postcode) & returns just the outward code. I will then use this outward code to reference the pgeocode DF with.
    if isinstance(value, str) and len(value) <= 8 and any(character.isdigit() for character in value):
        outward_code = value[:-3]
        return outward_code
    else:
        return np.nan

In [302]:
df['outwardCode'] = df['locationName'].apply(postcode_breakdown)
df['nonPostcodeValues'] = df["locationName"].where(pd.isna(df["outwardCode"]))
df


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,outwardCode,nonPostcodeValues
0,54419415,500373,QA,NaN,NaN,Data Administrator Apprentice,CV12AQ,14000.0,16000.0,GBP,18/03/2025,2025-02-04,About Premier Solutions Ltd: Premier Solutions...,57,https://www.reed.co.uk/jobs/data-administrator...,CV1,NaN
1,54494579,666205,Stark Building Materials UK,NaN,NaN,HR Data & Process Support Specialist,Coventry,NaN,NaN,NaN,17/03/2025,2025-02-17,We are seeking a motivated and detail-oriented...,35,https://www.reed.co.uk/jobs/hr-data-process-su...,NaN,Coventry
2,54599030,429392,Searchability,NaN,NaN,Data Scientist,Birmingham,45000.0,70000.0,GBP,17/04/2025,2025-03-06,"Data Scientist Key Highlights: Salary: 45,000 ...",29,https://www.reed.co.uk/jobs/data-scientist/545...,NaN,Birmingham
3,54405886,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham | Hybrid - 3 Days On...,69,https://www.reed.co.uk/jobs/data-scientist/544...,NaN,Birmingham
4,54404538,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham 3 days a week We are...,78,https://www.reed.co.uk/jobs/data-scientist/544...,NaN,Birmingham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414,54563954,674264,Appcast Enterprise,NaN,NaN,Litigation Handler/Solicitor,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/litigation-handler...,NaN,Tamworth
2415,54563961,674264,Appcast Enterprise,NaN,NaN,Senior Underwriter,Birmingham,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Gallagher isn't just a company; i...,0,https://www.reed.co.uk/jobs/senior-underwriter...,NaN,Birmingham
2416,54563922,674264,Appcast Enterprise,NaN,NaN,Litigation Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,1,https://www.reed.co.uk/jobs/litigation-handler...,NaN,Tamworth
2417,54563259,674264,Appcast Enterprise,NaN,NaN,Motor Claims Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/motor-claims-handl...,NaN,Tamworth


In [303]:
df_postcode = df[df['outwardCode'].notna()]
df_postcode

df_non_postcode = df[df['outwardCode'].isna()]
df_non_postcode

,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,outwardCode,nonPostcodeValues
1,54494579,666205,Stark Building Materials UK,NaN,NaN,HR Data & Process Support Specialist,Coventry,NaN,NaN,NaN,17/03/2025,2025-02-17,We are seeking a motivated and detail-oriented...,35,https://www.reed.co.uk/jobs/hr-data-process-su...,NaN,Coventry
2,54599030,429392,Searchability,NaN,NaN,Data Scientist,Birmingham,45000.0,70000.0,GBP,17/04/2025,2025-03-06,"Data Scientist Key Highlights: Salary: 45,000 ...",29,https://www.reed.co.uk/jobs/data-scientist/545...,NaN,Birmingham
3,54405886,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham | Hybrid - 3 Days On...,69,https://www.reed.co.uk/jobs/data-scientist/544...,NaN,Birmingham
4,54404538,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham 3 days a week We are...,78,https://www.reed.co.uk/jobs/data-scientist/544...,NaN,Birmingham
5,54436172,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Derby,55000.0,65000.0,GBP,20/03/2025,2025-02-06,Data Scientist - Hybrid (Derby) We are working...,55,https://www.reed.co.uk/jobs/data-scientist/544...,NaN,Derby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414,54563954,674264,Appcast Enterprise,NaN,NaN,Litigation Handler/Solicitor,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/litigation-handler...,NaN,Tamworth
2415,54563961,674264,Appcast Enterprise,NaN,NaN,Senior Underwriter,Birmingham,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Gallagher isn't just a company; i...,0,https://www.reed.co.uk/jobs/senior-underwriter...,NaN,Birmingham
2416,54563922,674264,Appcast Enterprise,NaN,NaN,Litigation Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,1,https://www.reed.co.uk/jobs/litigation-handler...,NaN,Tamworth
2417,54563259,674264,Appcast Enterprise,NaN,NaN,Motor Claims Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/motor-claims-handl...,NaN,Tamworth


In [304]:

df_postcode_merged = pd.merge(df_postcode, df_geo_unique, on='outwardCode', how='left')
df_postcode_merged = df_postcode_merged.drop(columns='nonPostcodeValues_x')
df_postcode_merged = df_postcode_merged.rename(columns={'nonPostcodeValues_y': 'nonPostcodeValues'})
df_postcode_merged


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,outwardCode,countryCode,nonPostcodeValues,countryName,countyName
0,54419415,500373,QA,NaN,NaN,Data Administrator Apprentice,CV12AQ,14000.0,16000.0,GBP,18/03/2025,2025-02-04,About Premier Solutions Ltd: Premier Solutions...,57,https://www.reed.co.uk/jobs/data-administrator...,CV1,GB,Coventry,England,West Midlands
1,53556034,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,B33DQ,30000.0,50000.0,GBP,15/04/2025,2024-09-11,Artificial Intelligence is used in every indus...,32,https://www.reed.co.uk/jobs/data-science-train...,B3,GB,Birmingham,England,West Midlands
2,54633075,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,DY11HP,30000.0,50000.0,GBP,23/04/2025,2025-03-12,Artificial Intelligence is used in every indus...,0,https://www.reed.co.uk/jobs/data-science-train...,DY1,GB,Dudley,England,West Midlands
3,53556204,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,LE19BG,30000.0,50000.0,GBP,21/04/2025,2024-09-11,Artificial Intelligence is used in every indus...,18,https://www.reed.co.uk/jobs/data-science-train...,LE1,GB,Leicester,England,Leicestershire
4,54050405,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,WS11PT,30000.0,50000.0,GBP,07/04/2025,2024-11-19,Artificial Intelligence is used in every indus...,12,https://www.reed.co.uk/jobs/data-science-train...,WS1,GB,Walsall,England,West Midlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,54406478,523861,High Profile Resourcing Ltd,NaN,NaN,HR Business Partner - Head Office,CV11DD,40000.0,60000.0,GBP,14/03/2025,2025-01-31,Human Resources Business Partner - corporate f...,39,https://www.reed.co.uk/jobs/hr-business-partne...,CV1,GB,Coventry,England,West Midlands
845,54574541,614244,Ibstock PLC,NaN,NaN,Category Manager / Senior Buyer,LE676HS,NaN,NaN,NaN,31/03/2025,2025-03-03,"To support our progress, we are currently recr...",2,https://www.reed.co.uk/jobs/category-manager-s...,LE67,GB,Ellistown,England,Leicestershire
846,54537093,613176,Ibstock Brick,NaN,NaN,SHE Coordinator,WS98BL,NaN,NaN,NaN,26/03/2025,2025-02-26,"WE ARE 200 years of history and over 2,300 emp...",8,https://www.reed.co.uk/jobs/she-coordinator/54...,WS9,GB,Stonnall,England,Staffordshire
847,54497660,106910,SF Recruitment,NaN,NaN,Administrator,LE191UZ,27000.0,28000.0,GBP,01/04/2025,2025-02-18,Administrator – 6-Month Fixed-Term Contract Lo...,78,https://www.reed.co.uk/jobs/administrator/5449...,LE19,GB,Leicester,England,Leicestershire


In [305]:
df_non_postcode_merged = pd.merge(df_non_postcode, df_geo_unique, on='nonPostcodeValues', how='left')
df_non_postcode_merged =df_non_postcode_merged.drop_duplicates(subset='jobId')
df_non_postcode_merged = df_non_postcode_merged.drop(columns='outwardCode_x')
df_non_postcode_merged = df_non_postcode_merged.rename(columns={'outwardCode_y': 'outwardCode'})
df_non_postcode_merged


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,nonPostcodeValues,countryCode,outwardCode,countryName,countyName
0,54494579,666205,Stark Building Materials UK,NaN,NaN,HR Data & Process Support Specialist,Coventry,NaN,NaN,NaN,17/03/2025,2025-02-17,We are seeking a motivated and detail-oriented...,35,https://www.reed.co.uk/jobs/hr-data-process-su...,Coventry,GB,CV1,England,West Midlands
1,54599030,429392,Searchability,NaN,NaN,Data Scientist,Birmingham,45000.0,70000.0,GBP,17/04/2025,2025-03-06,"Data Scientist Key Highlights: Salary: 45,000 ...",29,https://www.reed.co.uk/jobs/data-scientist/545...,Birmingham,GB,B1,England,West Midlands
19,54405886,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham | Hybrid - 3 Days On...,69,https://www.reed.co.uk/jobs/data-scientist/544...,Birmingham,GB,B1,England,West Midlands
37,54404538,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Birmingham,40000.0,50000.0,GBP,14/03/2025,2025-01-31,Data Scientist Birmingham 3 days a week We are...,78,https://www.reed.co.uk/jobs/data-scientist/544...,Birmingham,GB,B1,England,West Midlands
55,54436172,331522,Harnham - Data & Analytics Recruitment,NaN,NaN,Data Scientist,Derby,55000.0,65000.0,GBP,20/03/2025,2025-02-06,Data Scientist - Hybrid (Derby) We are working...,55,https://www.reed.co.uk/jobs/data-scientist/544...,Derby,GB,DE1,England,Derbyshire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,54563954,674264,Appcast Enterprise,NaN,NaN,Litigation Handler/Solicitor,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/litigation-handler...,Tamworth,NaN,NaN,NaN,NaN
8828,54563961,674264,Appcast Enterprise,NaN,NaN,Senior Underwriter,Birmingham,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Gallagher isn't just a company; i...,0,https://www.reed.co.uk/jobs/senior-underwriter...,Birmingham,GB,B1,England,West Midlands
8846,54563922,674264,Appcast Enterprise,NaN,NaN,Litigation Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,1,https://www.reed.co.uk/jobs/litigation-handler...,Tamworth,NaN,NaN,NaN,NaN
8847,54563259,674264,Appcast Enterprise,NaN,NaN,Motor Claims Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/motor-claims-handl...,Tamworth,NaN,NaN,NaN,NaN


In [306]:
df_export = pd.concat([df_postcode_merged, df_non_postcode_merged])

df_export

,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,outwardCode,countryCode,nonPostcodeValues,countryName,countyName
0,54419415,500373,QA,NaN,NaN,Data Administrator Apprentice,CV12AQ,14000.0,16000.0,GBP,18/03/2025,2025-02-04,About Premier Solutions Ltd: Premier Solutions...,57,https://www.reed.co.uk/jobs/data-administrator...,CV1,GB,Coventry,England,West Midlands
1,53556034,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,B33DQ,30000.0,50000.0,GBP,15/04/2025,2024-09-11,Artificial Intelligence is used in every indus...,32,https://www.reed.co.uk/jobs/data-science-train...,B3,GB,Birmingham,England,West Midlands
2,54633075,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,DY11HP,30000.0,50000.0,GBP,23/04/2025,2025-03-12,Artificial Intelligence is used in every indus...,0,https://www.reed.co.uk/jobs/data-science-train...,DY1,GB,Dudley,England,West Midlands
3,53556204,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,LE19BG,30000.0,50000.0,GBP,21/04/2025,2024-09-11,Artificial Intelligence is used in every indus...,18,https://www.reed.co.uk/jobs/data-science-train...,LE1,GB,Leicester,England,Leicestershire
4,54050405,582327,ITOL Recruitment,NaN,NaN,Data Science Trainee,WS11PT,30000.0,50000.0,GBP,07/04/2025,2024-11-19,Artificial Intelligence is used in every indus...,12,https://www.reed.co.uk/jobs/data-science-train...,WS1,GB,Walsall,England,West Midlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,54563954,674264,Appcast Enterprise,NaN,NaN,Litigation Handler/Solicitor,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/litigation-handler...,NaN,NaN,Tamworth,NaN,NaN
8828,54563961,674264,Appcast Enterprise,NaN,NaN,Senior Underwriter,Birmingham,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Gallagher isn't just a company; i...,0,https://www.reed.co.uk/jobs/senior-underwriter...,B1,GB,Birmingham,England,West Midlands
8846,54563922,674264,Appcast Enterprise,NaN,NaN,Litigation Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,1,https://www.reed.co.uk/jobs/litigation-handler...,NaN,NaN,Tamworth,NaN,NaN
8847,54563259,674264,Appcast Enterprise,NaN,NaN,Motor Claims Handler,Tamworth,NaN,NaN,NaN,12/04/2025,2025-03-01,Introduction Join our growing team of dedicate...,0,https://www.reed.co.uk/jobs/motor-claims-handl...,NaN,NaN,Tamworth,NaN,NaN


In [307]:
new_col_order = ['jobId', 'employerId', 'employerName', 'employerProfileId',
                'employerProfileName', 'jobTitle', 'locationName', 'minimumSalary',
                'maximumSalary', 'currency', 'date', 'expirationDate', 'applications', 'outwardCode',
                'countryCode', 'nonPostcodeValues', 'countryName', 'countyName', 'jobUrl', 'jobDescription']

In [308]:
df_export = df_export[new_col_order]

In [312]:
today = datetime.today().strftime('%d.%m.%Y')
print(today)

12.03.2025


In [314]:
df_export.to_excel(f'reed_{keywords}_jobs_{today}.xlsx')